In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\HARSH\Desktop\Ecommerce Sales Analytics\data\amazon_sales_data 2025.csv")
df.head()


,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


In [2]:
df.info()
df.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Order ID           250 non-null    object
 1   Date               250 non-null    object
 2   Product            250 non-null    object
 3   Category           250 non-null    object
 4   Price              250 non-null    int64 
 5   Quantity           250 non-null    int64 
 6   Total Sales        250 non-null    int64 
 7   Customer Name      250 non-null    object
 8   Customer Location  250 non-null    object
 9   Payment Method     250 non-null    object
 10  Status             250 non-null    object
dtypes: int64(3), object(8)
memory usage: 21.6+ KB


Order ID             0
Date                 0
Product              0
Category             0
Price                0
Quantity             0
Total Sales          0
Customer Name        0
Customer Location    0
Payment Method       0
Status               0
dtype: int64

In [3]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.columns



Index(['order_id', 'date', 'product', 'category', 'price', 'quantity',
       'total_sales', 'customer_name', 'customer_location', 'payment_method',
       'status'],
      dtype='object')

In [4]:
df['date'] = pd.to_datetime(df['date'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           250 non-null    object        
 1   date               250 non-null    datetime64[ns]
 2   product            250 non-null    object        
 3   category           250 non-null    object        
 4   price              250 non-null    int64         
 5   quantity           250 non-null    int64         
 6   total_sales        250 non-null    int64         
 7   customer_name      250 non-null    object        
 8   customer_location  250 non-null    object        
 9   payment_method     250 non-null    object        
 10  status             250 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 21.6+ KB


C:\Users\HARSH\AppData\Local\Temp\ipykernel_20112\66213139.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


In [6]:
df['date'] = pd.to_datetime(df['date'], format='mixed')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           250 non-null    object        
 1   date               250 non-null    datetime64[ns]
 2   product            250 non-null    object        
 3   category           250 non-null    object        
 4   price              250 non-null    int64         
 5   quantity           250 non-null    int64         
 6   total_sales        250 non-null    int64         
 7   customer_name      250 non-null    object        
 8   customer_location  250 non-null    object        
 9   payment_method     250 non-null    object        
 10  status             250 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 21.6+ KB


In [8]:
df['status'].value_counts()


status
Completed    88
Pending      85
Cancelled    77
Name: count, dtype: int64

In [9]:
df = df[df['status'].str.lower().isin(['completed', 'delivered', 'success'])]


In [10]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()


In [11]:
df['day'] = df['date'].dt.day


In [12]:
(df['total_sales'] - (df['price'] * df['quantity'])).describe()


count    88.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

In [13]:
df.to_csv("../data/cleaned_amazon_sales.csv", index=False)


In [15]:
#RFM ANALYSIS (CORE OF THE PROJECT

In [16]:
import datetime as dt

snapshot_date = df['date'].max() + dt.timedelta(days=1)

rfm = df.groupby('customer_name').agg({
    'date': lambda x: (snapshot_date - x.max()).days,
    'order_id': 'nunique',
    'total_sales': 'sum'
})

rfm.columns = ['recency', 'frequency', 'monetary']
rfm.head()


,recency,frequency,monetary
customer_name,,,
Chris White,63,7,2110
Daniel Harris,250,6,2670
David Lee,63,3,4550
Emily Johnson,62,9,9855
Emma Clark,92,16,20720


In [18]:
rfm['r'] = pd.qcut(rfm['recency'], 4, labels=[4,3,2,1])
rfm['f'] = pd.qcut(rfm['frequency'], 4, labels=[1,2,3,4])
rfm['m'] = pd.qcut(rfm['monetary'], 4, labels=[1,2,3,4])

rfm['rfm_score'] = rfm[['r','f','m']].sum(axis=1)


In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['recency','frequency','monetary']])

kmeans = KMeans(n_clusters=4, random_state=42)
rfm['segment'] = kmeans.fit_predict(rfm_scaled)


In [20]:
final_df = df.merge(
    rfm[['segment']],
    left_on='customer_name',
    right_index=True
)

final_df.to_csv("../data/final_amazon_sales.csv", index=False)
